# White Noise: Metadata Cleaning

## 1. Explaining the Problem
At last, I have my manually annotated data in the `labelled.csv` file, my automatically labelled data in the `unlabelled.csv` file, and the metadata for all bill summaries in `bill_metadata_house.jsonl`, `bill_metadata_senate.jsonl`, and `bill_metadata_rogue.jsonl`. The prospective steps for the metadata cleaning procedure are quite straightforward. First, I must re-structure the `.jsonl` files into a cleaning-friendly format - i.e., a `pandas` `DataFrame`. Second, I must join the data for the House and the Senate, plus the rogue metadata I retrieved thanks to the diagnostics file, in a single data set. Next, it will be time to attach `labelled.csv` and `unlabelled.csv` in a single `DataFrame` object. Then, I will merge the metadata for each single bill with the main `DataFrame`, exploiting unique values such as bill numbers, US Congress number, and US Congress branch identifiers. Finally, I will save this definitive version of the dataset, ready for statistical and graphical analysis, into a new `white_noise.csv` document.

In [1]:
# Packages for handling .jsonl and .csv files
import jsonlines
import csv

# Package for data cleaning
import pandas as pd

## 2. Unpacking Pandora's Box

I start by re-structuring the metadata regarding House bills into a `DataFrame` object.

In [2]:
# I first create an empty list to store all lines from the "bill_metadata_house.jsonl" file
house_metadata = []

# I open the .jsonl file, and read it with the following helper function
with jsonlines.open("bill_metadata_house.jsonl") as r:
    
    # I loop over each line...
    for line in r:
        
        # ...and I append it to the originally empty list!
        house_metadata.append(line)

# I now turn the list of lines - i.e., dictionaries - into a pandas DataFrame object
meta_house = pd.DataFrame(house_metadata)

In [3]:
# I check the first few lines of the dataset to assess if this cleaning step went smoothly

meta_house.head()

,congress,bill_number,bill_type,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
0,111,6523,hr,Armed Forces and National Security,IKE,SKELTON,MO,D
1,111,6561,hr,Social Sciences and History,Laura,Richardson,CA,D
2,111,81,hr,Public Lands and Natural Resources,Madeleine,Bordallo,GU,D
3,111,6533,hr,"Science, Technology, Communications",MIKE,DOYLE,PA,D
4,111,6510,hr,Public Lands and Natural Resources,SHEILA,JACKSON LEE,TX,D


In [4]:
# I check the last few lines of the dataset to assess if this cleaning step went smoothly

meta_house.tail()

,congress,bill_number,bill_type,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
13908,115,123,hr,Housing and Community Development,Al,Green,TX,D
13909,115,152,hr,Government Operations and Politics,Al,Green,TX,D
13910,115,149,hr,Housing and Community Development,Al,Green,TX,D
13911,115,125,hr,Housing and Community Development,Al,Green,TX,D
13912,115,122,hr,Labor and Employment,Al,Green,TX,D


In [5]:
# I check the DataFrame object's shape to assess if this cleaning step went smoothly

meta_house.shape

(13913, 8)

I have 13913 rows - i.e., House bills - and eight columns - i.e., the identifiers, and the information about the bill's policy area, and the bill's sponsor's name, surname, state of election, and party - into the `DataFrame` object. Taking the missing documents that I retrieved thanks to the diagnostics into account, it appears everything went perfectly! Now, I turn to the metadata concerning Senate bills.

In [6]:
# I first create an empty list to store all lines from the "bill_metadata_senate.jsonl" file
senate_metadata = []

# I open the .jsonl file, and read it with the following helper function
with jsonlines.open("bill_metadata_senate.jsonl") as r:
    
    # I loop over each line...
    for line in r:
        
        # ...and I append it to the originally empty list!
        senate_metadata.append(line)

# I now turn the list of lines - i.e., dictionaries - into a pandas DataFrame object
meta_senate = pd.DataFrame(senate_metadata)

In [7]:
# I check the first few lines of the dataset to assess if this cleaning step went smoothly

meta_senate.head()

,congress,bill_number,bill_type,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
0,111,841,s,Transportation and Public Works,JOHN,KERRY,MA,D
1,111,4036,s,Finance and Financial Sector,CHRISTOPHER,DODD,CT,D
2,111,3903,s,Native Americans,TOM,UDALL,NM,D
3,111,3874,s,Environmental Protection,BARBARA,BOXER,CA,D
4,111,3592,s,Government Operations and Politics,SAXBY,CHAMBLISS,GA,R


In [8]:
# I check the last few lines of the dataset to assess if this cleaning step went smoothly

meta_senate.tail()

,congress,bill_number,bill_type,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
7826,115,15,s,International Affairs,Dean,Heller,NV,R
7827,115,16,s,Finance and Financial Sector,Rand,Paul,KY,R
7828,115,14,s,Congress,Dean,Heller,NV,R
7829,115,13,s,Taxation,Sheldon,Whitehouse,RI,D
7830,115,11,s,International Affairs,Dean,Heller,NV,R


In [9]:
# I check the DataFrame object's shape to assess if this cleaning step went smoothly

meta_senate.shape

(7831, 8)

I have 7831 rows - i.e., House bills - and eight columns - i.e., the identifiers, and the information about the bill's policy area, and the bill's sponsor's name, surname, state of election, and party - into the `DataFrame` object. Taking the missing documents that I retrieved thanks to the diagnostics into account, it appears everything went perfectly! Now, I turn to the missing metadata retrieved thanks to the diagnostics file.

In [10]:
# I first create an empty list to store all lines from the "bill_metadata_rogue.jsonl" file
rogue_metadata = []

# I open the .jsonl file, and read it with the following helper function
with jsonlines.open("bill_metadata_rogue.jsonl") as r:
    
    # I loop over each line...
    for line in r:
        
        # ...and I append it to the originally empty list!
        rogue_metadata.append(line)

# I now turn the list of lines - i.e., dictionaries - into a pandas DataFrame object
meta_rogue = pd.DataFrame(rogue_metadata)

# I add an empty column which signals that the data concerning the bills' policy areas are missing
meta_rogue["policy_area"] = None

In [11]:
# I check the first few lines of the dataset to assess if this cleaning step went smoothly

meta_rogue.head()

,congress,bill_number,bill_type,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party,policy_area
0,115,7162,hr,RICHARD,NOLAN,MN,D,None
1,115,7134,hr,J.,Correa,CA,D,None
2,115,7135,hr,DON,YOUNG,AK,R,None
3,115,6895,hr,Darren,Soto,FL,D,None
4,115,6829,hr,Daniel,Kildee,MI,D,None


In [12]:
# I check the last few lines of the dataset to assess if this cleaning step went smoothly

meta_rogue.tail()

,congress,bill_number,bill_type,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party,policy_area
71,115,559,s,Dianne,Feinstein,CA,D,None
72,115,460,s,Mazie,Hirono,HI,D,None
73,115,433,s,Robert,Menendez,NJ,D,None
74,115,388,s,Sherrod,Brown,OH,D,None
75,115,187,s,Charles,Schumer,NY,D,None


In [13]:
# I check the DataFrame object's shape to assess if this cleaning step went smoothly

meta_rogue.shape

(76, 8)

In [14]:
# I print the DataFrame objects' total length to assess whether I correctly retrieved metadata on all the bills of interest.

print(f"I retrieved metadata on {len(meta_house)+len(meta_senate)+len(meta_rogue)} US Congress bills.")

I retrieved metadata on 21820 US Congress bills.


I have 76 rows - i.e., House bills - and eight columns - i.e., the identifiers, and the bill's sponsor's name, surname, state of election, and party - into the `DataFrame` object. An additional column is geared towards indicating that the information about policy areas is missing in the bills that I could not originally retrieve. It appears that I correctly retrieved metadata on all the 21820 US Congress bills of interest, and that everything went perfectly! Next, I turn to joining all the metadata into one single `DataFrame` object. 

## 3. Joining the Metadata

In [15]:
# I concatenate the three DataFrames vertically, ignoring their respective indexes because they do not provide any
# useful information, by setting "ignore_index" argument to "True"

meta_final = pd.concat([meta_house, meta_senate, meta_rogue], ignore_index = True)

In [16]:
# I check the first few lines of the dataset to assess if this cleaning step went smoothly

meta_final.head()

,congress,bill_number,bill_type,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
0,111,6523,hr,Armed Forces and National Security,IKE,SKELTON,MO,D
1,111,6561,hr,Social Sciences and History,Laura,Richardson,CA,D
2,111,81,hr,Public Lands and Natural Resources,Madeleine,Bordallo,GU,D
3,111,6533,hr,"Science, Technology, Communications",MIKE,DOYLE,PA,D
4,111,6510,hr,Public Lands and Natural Resources,SHEILA,JACKSON LEE,TX,D


In [17]:
# I check the last few lines of the dataset to assess if this cleaning step went smoothly

meta_final.tail()

,congress,bill_number,bill_type,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
21815,115,559,s,None,Dianne,Feinstein,CA,D
21816,115,460,s,None,Mazie,Hirono,HI,D
21817,115,433,s,None,Robert,Menendez,NJ,D
21818,115,388,s,None,Sherrod,Brown,OH,D
21819,115,187,s,None,Charles,Schumer,NY,D


In [18]:
# I check the DataFrame object's shape to assess if this cleaning step went smoothly

meta_final.shape

(21820, 8)

I have 21820 rows - i.e., bills - and eight columns - i.e., the identifiers, and the information about the bill's policy area, and the bill's sponsor's name, surname, state of election, and party - into the `DataFrame` object. It appears everything went perfectly! Before turning to concatenating `labelled.csv` and `unlabelled.csv`, I save the `DataFrame` as the `metadata.csv` file to keep it as a backup.

In [19]:
# I save the "meta_final" DataFrame as "metadata.csv". I employ "|" as a separator to prevent the pd methods from confusing
# colons or semi-colons within the texts with the actual separators. I set the "index" argument to false, because the indexes
# are completely meaningless and do not need to be saved.

meta_final.to_csv("metadata.csv", sep = "|", index = False)

## 4. Joining the Main Data

I start by importing the `labelled.csv` and `unlabelled.csv` files as `DataFrame` objects.

In [20]:
# I import the "labelled.csv" and "unlabelled.csv" data sets as DataFrame objects within the Python environment.
# I crucially specify the "|" separator, because employing colons or semi-colons causes conflicts with the summaries' contents.

labelled = pd.read_csv("labelled.csv", sep = "|") # "labelled.csv"
unlabelled = pd.read_csv("unlabelled.csv", sep = "|") # "unlabelled.csv"

In [21]:
# I check the first few lines of the "labelled" dataset to assess if this cleaning step went smoothly

labelled.head()

,congress,bill_number,bill_type,text,economic,socio_cultural
0,115,1308,hr,Frank and Jeanne Moore Wild Steelhead Speci...,Non-Economic,Socio-Cultural
1,115,4105,hr,This bill extends funding through FY2022 for...,Economic,Non-Socio-Cultural
2,115,3691,s,Expanding Transparency of Information and S...,Non-Economic,Socio-Cultural
3,111,1994,hr,Citizen Soldier Equality Act of 2009 - Requi...,Economic,Socio-Cultural
4,111,883,hr,"Amends the Internal Revenue Code to repeal, e...",Economic,Socio-Cultural


In [22]:
# I check the "labelled" object's shape to assess if this cleaning step went smoothly

labelled.shape

(2200, 6)

In [23]:
# I check the first few lines of the "unlabelled" dataset to assess if this cleaning step went smoothly

unlabelled.head()

,congress,bill_number,bill_type,text,economic,socio_cultural
0,115,5405,hr,This bill amends the Agricultural Act of 201...,Economic,Non-Socio-Cultural
1,115,1980,s,Renewable Chemicals Act of 2017 This bil...,Economic,Socio-Cultural
2,115,4769,hr,Helping Americans Seek Treatment Act of 201...,Non-Economic,Socio-Cultural
3,111,1558,s,Travel Reimbursement for Inactive Duty Trai...,Economic,Non-Socio-Cultural
4,115,3932,hr,Healthcare Expenditures for Low-income Popu...,Economic,Socio-Cultural


In [24]:
# I check the "unlabelled" object's shape to assess if this cleaning step went smoothly

unlabelled.shape

(19620, 6)

I have 2200, and 19620 rows - i.e., bills - and six columns - i.e., identifiers, texts, and the two labels - into the `DataFrame` objects. It appears everything went perfectly! Now, I can concatenate them in a single dataset.

In [25]:
# I concatenate the two DataFrames vertically, ignoring their respective indexes because they do not provide any
# useful information, by setting "ignore_index" argument to "True".

main = pd.concat([labelled, unlabelled], ignore_index = True)

In [26]:
# I check the first few lines of the dataset to assess if this cleaning step went smoothly

main.head()

,congress,bill_number,bill_type,text,economic,socio_cultural
0,115,1308,hr,Frank and Jeanne Moore Wild Steelhead Speci...,Non-Economic,Socio-Cultural
1,115,4105,hr,This bill extends funding through FY2022 for...,Economic,Non-Socio-Cultural
2,115,3691,s,Expanding Transparency of Information and S...,Non-Economic,Socio-Cultural
3,111,1994,hr,Citizen Soldier Equality Act of 2009 - Requi...,Economic,Socio-Cultural
4,111,883,hr,"Amends the Internal Revenue Code to repeal, e...",Economic,Socio-Cultural


In [27]:
# I check the last few lines of the dataset to assess if this cleaning step went smoothly

main.tail()

,congress,bill_number,bill_type,text,economic,socio_cultural
21815,111,1063,hr,Amends the Energy Independence and Security ...,Non-Economic,Socio-Cultural
21816,111,2839,s,Torture Victims Relief Reauthorization Act o...,Economic,Socio-Cultural
21817,111,834,hr,Ramos and Compean Justice Act of 2009 - Amen...,Non-Economic,Socio-Cultural
21818,115,1935,s,Tribal Tax and Investment Reform Act of 201...,Economic,Socio-Cultural
21819,111,1940,hr,Wellness Trust Act - Amends the Public Healt...,Economic,Socio-Cultural


In [28]:
# I check the DataFrame object's shape to assess if this cleaning step went smoothly

main.shape

(21820, 6)

I have 21820 rows - i.e., bills - and six columns - i.e., identifiers, texts, and the two labels - into the `DataFrame` object. It appears everything went perfectly! I can finally merge the main data with the bill metadata, with the `pandas` method `merge`.

## 5. Generating the Dataset's Final Version

In [29]:
# I convert all the numeric columns I wish to employ as keys for merging to the consistent "string" type in both datasets.

# Main dataset
main["congress"] = main["congress"].astype(str)
main["bill_number"] = main["bill_number"].astype(str)

# Metadata dataset
meta_final["congress"] = meta_final["congress"].astype(str)
meta_final["bill_number"] = meta_final["bill_number"].astype(str)

# I merge the "main" and "meta_final" dataset by exploiting each bill's congress number, numeric identifier, and chamber of
# introduction, which combined yield unique instances for identification. 

white_noise = pd.merge(main, meta_final, on = ["congress", "bill_number", "bill_type"], how = "inner")

In [30]:
# I check the first few lines of the dataset to assess if this cleaning step went smoothly

white_noise.head()

,congress,bill_number,bill_type,text,economic,socio_cultural,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
0,115,1308,hr,Frank and Jeanne Moore Wild Steelhead Speci...,Non-Economic,Socio-Cultural,Public Lands and Natural Resources,PETER,DEFAZIO,OR,D
1,115,4105,hr,This bill extends funding through FY2022 for...,Economic,Non-Socio-Cultural,Health,Brenda,Lawrence,MI,D
2,115,3691,s,Expanding Transparency of Information and S...,Non-Economic,Socio-Cultural,Environmental Protection,Tammy,Duckworth,IL,D
3,111,1994,hr,Citizen Soldier Equality Act of 2009 - Requi...,Economic,Socio-Cultural,Armed Forces and National Security,Geoff,Davis,KY,R
4,111,883,hr,"Amends the Internal Revenue Code to repeal, e...",Economic,Socio-Cultural,Taxation,PETER,KING,NY,R


In [31]:
# I check the last few lines of the dataset to assess if this cleaning step went smoothly

white_noise.tail()

,congress,bill_number,bill_type,text,economic,socio_cultural,policy_area,sponsor_name,sponsor_lastname,sponsor_state,sponsor_party
21815,111,1063,hr,Amends the Energy Independence and Security ...,Non-Economic,Socio-Cultural,Energy,Jeb,Hensarling,TX,R
21816,111,2839,s,Torture Victims Relief Reauthorization Act o...,Economic,Socio-Cultural,International Affairs,Amy,Klobuchar,MN,D
21817,111,834,hr,Ramos and Compean Justice Act of 2009 - Amen...,Non-Economic,Socio-Cultural,Crime and Law Enforcement,Ted,Poe,TX,R
21818,115,1935,s,Tribal Tax and Investment Reform Act of 201...,Economic,Socio-Cultural,Native Americans,Jerry,Moran,KS,R
21819,111,1940,hr,Wellness Trust Act - Amends the Public Healt...,Economic,Socio-Cultural,Health,Doris,Matsui,CA,D


In [32]:
# I check the DataFrame object's shape to assess if this cleaning step went smoothly

white_noise.shape

(21820, 11)

I have 21820 rows - i.e., bills - and eleven columns - i.e., the three identifiers, the summary texts, the two labels, and the five types of information I retrieved as metadata - into the `DataFrame` object. It appears everything went perfectly! To conclude, I save this definitive version of the dataset, ready for statistical and graphical analysis, into a new `white_noise.csv` document.

## 6. Wrapping Up

The data cleaning journey ends here. It is time to carry out my statistical and graphic analysis, and start writing the final report!

In [33]:
# I save the "white_noise" DataFrame as "white_noise.csv". I employ "|" as a separator to prevent the pd methods from confusing
# colons or semi-colons within the texts with the actual separators. I set the "index" argument to false, because the indexes
# are completely meaningless and do not need to be saved.

white_noise.to_csv("white_noise.csv", sep = "|", index = False)